# Online Retail RFM Analysis

## Part 1: EDA and Data Cleaning

In this project we'll do RFM analysis from a 2-years-worth of data in online retail. We're taking a dataset from Online Retail II UCI Machine Learning Repository, containing all the transactions occuring for a UK-based and registered, non-store online retail between 01/12/2009 to 09/12/2011. Many customers of the company are wholesalers.

This data consists of eight columns:

* `InvoiceNo` : Invoice Number; Code starting with 'C' indicates a cancellation,
* `StockCode` : Product (item) code; 5-digit integral number assigned to each distinct product,
* `Description` : Product (item) name,
* `Quantity` : The number of product (item) bought per transaction,
* `InvoiceDate` : Invoice date and time,
* `UnitPrice` : Product price per unit (in poundsterling),
* `Customer ID` : Customer number as identifier, and
* `Country` : Name of country where the customer resides.

In [1]:
import pandas as pd

# ## (Option 1) Separating the file into csv per sheets
# # for sheet_name, df in pd.read_excel(
# #     'online_retail_II.xlsx', index_col=0, sheet_name=None).items():
# #     df.to_csv(f'online_retail_II_{sheet_name}.csv', index=False, encoding='utf-8')

## (Option 2) Combining the sheets into a single csv
# combined_dfs = []

# for sheet_name, df in pd.read_excel(
#     'online_retail_II.xlsx',sheet_name=None).items():
    
#     # Version 1: Original append
#     combined_dfs.append(df)

# combined_dfs = pd.concat(combined_dfs)
# combined_dfs.to_csv('online_retail_II.csv', index=False)

In [2]:
import pandas as pd
combined_dfs = pd.read_csv('online_retail_II.csv')

In [3]:
combined_dfs.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


We're making a new column of `revenue: Quantity * Price` for data cleaning up to prediction.

In [4]:
combined_dfs['revenue'] = combined_dfs['Price'] * combined_dfs['Quantity']

In [5]:
combined_dfs

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,14.85


In [6]:
combined_dfs.shape

(1067371, 9)

In [7]:
combined_dfs.describe()

,Quantity,Price,Customer ID,revenue
count,1.067371e+06,1.067371e+06,824364.000000,1.067371e+06
mean,9.938898e+00,4.649388e+00,15324.638504,1.806987e+01
std,1.727058e+02,1.235531e+02,1697.464450,2.924202e+02
min,-8.099500e+04,-5.359436e+04,12346.000000,-1.684696e+05
25%,1.000000e+00,1.250000e+00,13975.000000,3.750000e+00
50%,3.000000e+00,2.100000e+00,15255.000000,9.900000e+00
75%,1.000000e+01,4.150000e+00,16797.000000,1.770000e+01
max,8.099500e+04,3.897000e+04,18287.000000,1.684696e+05


## Missing Values

To make sure we run the machine learning model into our data smoothly, we need to make sure there's no missing values.

In [8]:
combined_dfs.isna().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
revenue             0
dtype: int64

In [9]:
combined_dfs.loc[combined_dfs['Customer ID'].isna()].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom,-0.00
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom,-0.00
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom,-0.00
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom,-0.00
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom,0.55


In [10]:
combined_dfs.loc[combined_dfs['Description'].isna()].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom,-0.0
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom,-0.0
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,NaN,United Kingdom,0.0
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.0,NaN,United Kingdom,0.0
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom,-0.0


In [11]:
combined_dfs.loc[combined_dfs['Customer ID'].isna(), 'Price'].value_counts()

Price
2.46       16081
2.51       14861
4.13       14804
1.66       12704
1.63       12027
           ...  
0.60           1
124.46         1
163.15         1
107.85         1
1714.17        1
Name: count, Length: 2155, dtype: int64

In [12]:
combined_dfs.loc[combined_dfs['Description'].isna(), 'Price'].value_counts()

Price
0.0    4382
Name: count, dtype: int64

We won't know one's customer behavior through RFM without knowing its `Customer ID` as an identifier and `Price` fort the monetary value (in the missing values of `Description`, all values have a price of `0.0`), so in this case we'll just drop them out.

In [13]:
combined_dfs = combined_dfs.dropna(subset=['Customer ID', 'Description'])

In [14]:
combined_dfs.isna().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
revenue        0
dtype: int64

## Checking Columns:

### `Invoice` -- Checking Returns and Invalid Invoices 

In [15]:
combined_dfs.sort_values(['Invoice']).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [16]:
combined_dfs.sort_values(['Invoice']).tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom,-9.13
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,-224.69
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom,-54.75
1067178,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,-6.25
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,-1.25


In [17]:
combined_dfs.sort_values('Price')

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
248583,513416,22423,REGENCY CAKESTAND 3 TIER,5,2010-06-24 12:34:00,0.00,13089.0,United Kingdom,0.00
392008,527084,22630,DOLLY GIRL LUNCH BOX,64,2010-10-14 15:33:00,0.00,14646.0,Netherlands,0.00
16107,490727,M,Manual,1,2009-12-07 16:38:00,0.00,17231.0,United Kingdom,0.00
612250,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,16,2011-02-10 13:08:00,0.00,17560.0,United Kingdom,0.00
945865,572893,21208,PASTEL COLOUR HONEYCOMB FAN,5,2011-10-26 14:36:00,0.00,18059.0,United Kingdom,0.00
...,...,...,...,...,...,...,...,...,...
135013,502263,M,Manual,1,2010-03-23 15:22:00,10953.50,12918.0,United Kingdom,10953.50
135012,C502262,M,Manual,-1,2010-03-23 15:20:00,10953.50,12918.0,United Kingdom,-10953.50
135014,C502264,M,Manual,-1,2010-03-23 15:24:00,10953.50,12918.0,United Kingdom,-10953.50
241824,C512770,M,Manual,-1,2010-06-17 16:52:00,25111.09,17399.0,United Kingdom,-25111.09


In [18]:
combined_dfs.loc[combined_dfs['Invoice'].str.startswith('C', na=False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia,-35.4
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia,-9.9
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia,-17.0
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia,-12.6
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia,-35.4


In [19]:
combined_dfs.loc[combined_dfs['Invoice'].str.startswith('C', na=False)].sort_values('Price', ascending=False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
748142,C556445,M,Manual,-1,2011-06-10 15:31:00,38970.00,15098.0,United Kingdom,-38970.00
241824,C512770,M,Manual,-1,2010-06-17 16:52:00,25111.09,17399.0,United Kingdom,-25111.09
135012,C502262,M,Manual,-1,2010-03-23 15:20:00,10953.50,12918.0,United Kingdom,-10953.50
135014,C502264,M,Manual,-1,2010-03-23 15:24:00,10953.50,12918.0,United Kingdom,-10953.50
342135,C522793,M,Manual,-1,2010-09-16 14:53:00,10468.80,14063.0,United Kingdom,-10468.80


In [20]:
combined_dfs.loc[combined_dfs['StockCode'] == 'M'].describe()

,Quantity,Price,Customer ID,revenue
count,1115.000000,1115.000000,1115.000000,1115.000000
mean,4.034978,427.243202,15233.362332,-166.056709
std,87.772339,1818.640981,1771.871015,1873.853572
min,-1350.000000,0.000000,12346.000000,-38970.000000
25%,-1.000000,0.850000,13634.000000,-20.300000
50%,1.000000,4.500000,15202.000000,1.650000
75%,2.000000,159.540000,16760.500000,10.500000
max,1600.000000,38970.000000,18283.000000,10953.500000


Some invoices have an item returned for a whopping thousands of dollars, way off than the rest of the data. These data also belong to a  `StockCode` of `M`, and `Description` of  `Manual`.

These data seems to not portray one's customer behavior. Even if we'll put it into account, it'll become outliers compared to other transactions, seeing that each product costs in range of 0-10 pounds. We'll clean it in the next section.

There's also another types of `StockCode` that may give us invalid data; take `D` and `POST`, for instance--they have one same `StockCode` as product identifier, yet the prices differ. 

The dataset tells us that `StockCode` returns a product identifier which contains at least 5 numbers. We need to check them further first in the column `Description`.

### `StockCode` and `Description`: Types of Transaction

In this column, we'll check what kind of transactions there are in the dataset.

In [21]:
## Searching for Stock Codes besides the format:
### 5 digits, optionally followed by an uppercase. e.g. 79323P
non_digit_stock_code = combined_dfs.loc[~(combined_dfs['StockCode']
                           .str.contains(r'^\d{5}(?:[A-Z])?$')), :]

non_digit_stock_code.value_counts(['StockCode'])

StockCode   
POST            2019
M               1115
15056BL          850
C2               259
79323LP          219
D                174
79323GR          113
ADJUST            61
BANK CHARGES      38
PADS              19
CRUK              16
DOT               16
TEST001           15
ADJUST2            3
SP1002             2
TEST002            1
Name: count, dtype: int64

In [22]:
non_digit_stock_code.value_counts(['StockCode', 'Description'])

StockCode     Description                        
POST          POSTAGE                                2019
M             Manual                                 1115
15056BL       EDWARDIAN PARASOL BLACK                 850
C2            CARRIAGE                                259
79323LP       LIGHT PINK CHERRY LIGHTS                219
D             Discount                                174
79323GR       GREEN CHERRY LIGHTS                     113
ADJUST        Adjustment by john on 26/01/2010 16      36
BANK CHARGES  Bank Charges                             35
ADJUST        Adjustment by john on 26/01/2010 17      22
PADS          PADS TO MATCH ALL CUSHIONS               19
CRUK          CRUK Commission                          16
DOT           DOTCOM POSTAGE                           16
TEST001       This is a test product.                  15
ADJUST2       Adjustment by Peter on Jun 25 2010        3
BANK CHARGES   Bank Charges                             3
ADJUST        Adjustme

After filtering, we have 5 products, `D` for discount, and the rest includes  other transactions:

* `M` as in `Manual` for manual input, 
* test products as in `TEST001` and `TEST002`, 
* payments like `BANK CHARGES` and `CRUK` for `CRUK Commission`, 
* up to delivery fee, as shown in both `POSTAGE` and also `CARRIAGE`. 

These are not relevant to the product transactions and some of them differs in extreme range. So in this case, we'll clean them all.

In [23]:
stock_codes_to_clean = ['M', 'POST', 'C2', 'CRUK', 'DOT', 'BANK CHARGES', 
                        'ADJUST', 'TEST001', 'ADJUST2', 'SP1002']

combined_dfs = combined_dfs.loc[~combined_dfs['StockCode'].isin(stock_codes_to_clean)]

Before we go, let's check customers who got discount.

In [24]:
customer_with_discount = (
    combined_dfs.loc[combined_dfs['StockCode'] == 'D']
        .sort_values('Price', ascending=False))
customer_with_discount.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
676052,C549452,D,Discount,-1,2011-04-08 14:17:00,1867.86,17940.0,United Kingdom,-1867.86
450818,C532255,D,Discount,-1,2010-11-11 12:50:00,1269.51,14088.0,United Kingdom,-1269.51
95554,C498420,D,Discount,-1,2010-02-18 16:25:00,947.08,14646.0,Netherlands,-947.08
399358,C527516,D,Discount,-1,2010-10-18 13:03:00,619.92,18102.0,United Kingdom,-619.92
318602,C520413,D,Discount,-1,2010-08-26 09:38:00,591.43,15498.0,United Kingdom,-591.43


In [25]:
cust_d_list = customer_with_discount['Customer ID'].unique()
cust_d_list

array([17940., 14088., 14646., 18102., 15498., 14911., 14912., 17389.,
       17340., 13694., 13316., 16029., 13408., 16013., 12843., 17450.,
       17841., 14527., 17949., 14800., 12830., 15299., 15044., 14031.,
       17581., 17032., 16422., 14441., 13027., 18277., 13137., 14159.,
       15796., 14593., 16655., 13634., 16684., 14878., 13564., 15911.,
       14994., 13693., 17858., 17377., 15838., 15901., 15494., 13798.,
       18167., 15607., 16672., 12346., 13206., 12931., 12901.])

In [26]:
cust_discount_number_of_transactions_per_date = (
    combined_dfs.loc[combined_dfs['Customer ID'].isin(cust_d_list)]
    .groupby(['Customer ID', 'InvoiceDate'])
    .agg({'Quantity' : lambda x: len(x[x > 0]),      # no_of_invoices
          'StockCode': lambda x: len(x[x == 'D']),   # no_of_discount_invoices
           'revenue' : lambda x: x.sum()             # revenue 
         })  
    .reset_index())
cust_discount_number_of_transactions_per_date.columns = [
    'Customer ID', 'InvoiceDate', 'no_of_invoices', 
    'no_of_discount_invoices', 'revenue']

cust_discount_number_of_transactions_per_date.sort_values('Customer ID', ascending=False)

,Customer ID,InvoiceDate,no_of_invoices,no_of_discount_invoices,revenue
3286,18277.0,2011-10-12 15:22:00,8,0,110.38
3285,18277.0,2011-01-25 12:34:00,0,0,-12.75
3284,18277.0,2010-11-16 16:59:00,0,0,-7.95
3283,18277.0,2010-11-07 15:52:00,19,0,303.21
3282,18277.0,2010-10-19 12:06:00,19,0,429.32
...,...,...,...,...,...
2,12346.0,2010-06-28 13:53:00,19,0,142.31
1,12346.0,2010-03-02 13:08:00,5,0,27.05
4,12346.0,2011-01-18 10:01:00,1,0,77183.60
5,12346.0,2011-01-18 10:17:00,0,0,-77183.60


In [27]:
cust_discount_total_transactions = (
    cust_discount_number_of_transactions_per_date
        .groupby(['Customer ID'])
        .agg({
            'no_of_invoices': 'sum',
            'no_of_discount_invoices' : 'sum',
            'revenue' : 'sum'
        })
)

cust_discount_total_transactions

,no_of_invoices,no_of_discount_invoices,revenue
Customer ID,,,
12346.0,26,1,169.360
12830.0,38,1,6748.400
12843.0,219,1,6326.110
12901.0,116,1,16308.940
12931.0,218,5,71422.670
13027.0,86,1,17191.200
13137.0,1441,1,7667.370
13206.0,581,1,8376.730
13316.0,175,1,12906.930


The discounts don't infer any invalid data to the customers, so we'll leave the data as is. 

### `Quantity`: Outlier Check

In [28]:
combined_dfs[combined_dfs['Quantity'] == 0].shape

(0, 9)

In [29]:
combined_dfs[combined_dfs['Quantity'] <= 0].sort_values('Quantity')[:10]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
1065883,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,-168469.60
587085,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,-77183.60
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom,-280.80
529729,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom,-280.80
359669,C524235,21088,SET/6 FRUIT SALAD PAPER CUPS,-7128,2010-09-28 11:02:00,0.08,14277.0,France,-570.24
359670,C524235,21096,SET/6 FRUIT SALAD PAPER PLATES,-7008,2010-09-28 11:02:00,0.13,14277.0,France,-911.04
359630,C524235,16047,POP ART PEN CASE & PENS,-5184,2010-09-28 11:02:00,0.08,14277.0,France,-414.72
359636,C524235,37340,MULTICOLOUR SPRING FLOWER MUG,-4992,2010-09-28 11:02:00,0.10,14277.0,France,-499.20
359653,C524235,85110,BLACK SILVER FLOWER T-LIGHT HOLDER,-4752,2010-09-28 11:02:00,0.07,14277.0,France,-332.64
359658,C524235,16046,TEATIME PEN CASE & PENS,-4608,2010-09-28 11:02:00,0.08,14277.0,France,-368.64


In [30]:
combined_dfs[combined_dfs['Quantity'] >= 0].sort_values('Quantity', ascending=False)[:10]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
1065882,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,168469.6
587080,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.6
90857,497946,37410,BLACK AND WHITE PAISLEY FLOWER MUG,19152,2010-02-15 11:57:00,0.10,13902.0,Denmark,1915.2
127166,501534,21099,SET/6 STRAWBERRY PAPER CUPS,12960,2010-03-17 13:09:00,0.10,13902.0,Denmark,1296.0
127168,501534,21091,SET/6 WOODLAND PAPER PLATES,12960,2010-03-17 13:09:00,0.10,13902.0,Denmark,1296.0
127169,501534,21085,SET/6 WOODLAND PAPER CUPS,12744,2010-03-17 13:09:00,0.10,13902.0,Denmark,1274.4
1027583,578841,84826,ASSTD DESIGN 3D PAPER STICKERS,12540,2011-11-25 15:57:00,0.00,13256.0,United Kingdom,0.0
127167,501534,21092,SET/6 STRAWBERRY PAPER PLATES,12480,2010-03-17 13:09:00,0.10,13902.0,Denmark,1248.0
135027,502269,21984,PACK OF 12 PINK PAISLEY TISSUES,10000,2010-03-23 15:36:00,0.25,17940.0,United Kingdom,2500.0
135030,502269,21981,PACK OF 12 WOODLAND TISSUES,10000,2010-03-23 15:36:00,0.25,17940.0,United Kingdom,2500.0


Even though many buyers are wholesalers, we can see that two transactions with highest quanitty got returned. Since they cancel out and it disrupts the data training later on, we'll just clean them.

In [31]:
invoices = ['C581484', 'C541433', '581483', '541431']
combined_dfs = combined_dfs.drop(
    combined_dfs[combined_dfs['Invoice'].isin(invoices)].index)

In [32]:
combined_dfs[combined_dfs['Quantity'] <= 0].sort_values('Quantity')[:10]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom,-280.80
529729,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom,-280.80
359669,C524235,21088,SET/6 FRUIT SALAD PAPER CUPS,-7128,2010-09-28 11:02:00,0.08,14277.0,France,-570.24
359670,C524235,21096,SET/6 FRUIT SALAD PAPER PLATES,-7008,2010-09-28 11:02:00,0.13,14277.0,France,-911.04
359630,C524235,16047,POP ART PEN CASE & PENS,-5184,2010-09-28 11:02:00,0.08,14277.0,France,-414.72
359636,C524235,37340,MULTICOLOUR SPRING FLOWER MUG,-4992,2010-09-28 11:02:00,0.10,14277.0,France,-499.20
359653,C524235,85110,BLACK SILVER FLOWER T-LIGHT HOLDER,-4752,2010-09-28 11:02:00,0.07,14277.0,France,-332.64
359658,C524235,16046,TEATIME PEN CASE & PENS,-4608,2010-09-28 11:02:00,0.08,14277.0,France,-368.64
359654,C524235,85160A,WHITE BIRD GARDEN DESIGN MUG,-4320,2010-09-28 11:02:00,0.13,14277.0,France,-561.60
359674,C524235,85184D,S/4 BLUE ROUND DECOUPAGE BOXES,-3936,2010-09-28 11:02:00,0.42,14277.0,France,-1653.12


In [33]:
combined_dfs[combined_dfs['Quantity'] >= 0].sort_values('Quantity', ascending=False)[:10]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
90857,497946,37410,BLACK AND WHITE PAISLEY FLOWER MUG,19152,2010-02-15 11:57:00,0.10,13902.0,Denmark,1915.2
127168,501534,21091,SET/6 WOODLAND PAPER PLATES,12960,2010-03-17 13:09:00,0.10,13902.0,Denmark,1296.0
127166,501534,21099,SET/6 STRAWBERRY PAPER CUPS,12960,2010-03-17 13:09:00,0.10,13902.0,Denmark,1296.0
127169,501534,21085,SET/6 WOODLAND PAPER CUPS,12744,2010-03-17 13:09:00,0.10,13902.0,Denmark,1274.4
1027583,578841,84826,ASSTD DESIGN 3D PAPER STICKERS,12540,2011-11-25 15:57:00,0.00,13256.0,United Kingdom,0.0
127167,501534,21092,SET/6 STRAWBERRY PAPER PLATES,12480,2010-03-17 13:09:00,0.10,13902.0,Denmark,1248.0
135028,502269,21982,PACK OF 12 SUKI TISSUES,10000,2010-03-23 15:36:00,0.25,17940.0,United Kingdom,2500.0
135027,502269,21984,PACK OF 12 PINK PAISLEY TISSUES,10000,2010-03-23 15:36:00,0.25,17940.0,United Kingdom,2500.0
135030,502269,21981,PACK OF 12 WOODLAND TISSUES,10000,2010-03-23 15:36:00,0.25,17940.0,United Kingdom,2500.0
135029,502269,21980,PACK OF 12 RED SPOTTY TISSUES,10000,2010-03-23 15:36:00,0.25,17940.0,United Kingdom,2500.0


For other data, considering that *many buyers are wholesalers*, which means they buy good amount of items in one transaction, we'll keep the data as is for now.

### `Price`: Outlier Check

In [34]:
combined_dfs.sort_values('Price', ascending=False)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
676052,C549452,D,Discount,-1,2011-04-08 14:17:00,1867.86,17940.0,United Kingdom,-1867.86
450818,C532255,D,Discount,-1,2010-11-11 12:50:00,1269.51,14088.0,United Kingdom,-1269.51
95554,C498420,D,Discount,-1,2010-02-18 16:25:00,947.08,14646.0,Netherlands,-947.08
748143,556446,22502,PICNIC BASKET WICKER 60 PIECES,1,2011-06-10 15:33:00,649.50,15098.0,United Kingdom,649.50
748132,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.50,15098.0,United Kingdom,38970.00
...,...,...,...,...,...,...,...,...,...
713074,553000,47566,PARTY BUNTING,4,2011-05-12 15:21:00,0.00,17667.0,United Kingdom,0.00
804784,561284,22167,OVAL WALL MIRROR DIAMANTE,1,2011-07-26 12:24:00,0.00,16818.0,United Kingdom,0.00
808373,561669,22960,JAM MAKING SET WITH JARS,11,2011-07-28 17:09:00,0.00,12507.0,Spain,0.00
979925,575579,22089,PAPER BUNTING VINTAGE PAISLEY,24,2011-11-10 11:49:00,0.00,13081.0,United Kingdom,0.00


In [35]:
combined_dfs[combined_dfs['Price'] == 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
4674,489825,22076,6 RIBBONS EMPIRE,12,2009-12-02 13:34:00,0.0,16126.0,United Kingdom,0.0
6781,489998,48185,DOOR MAT FAIRY CAKE,2,2009-12-03 11:19:00,0.0,15658.0,United Kingdom,0.0
18738,490961,22065,CHRISTMAS PUDDING TRINKET POT,1,2009-12-08 15:25:00,0.0,14108.0,United Kingdom,0.0
18739,490961,22142,CHRISTMAS CRAFT WHITE FAIRY,12,2009-12-08 15:25:00,0.0,14108.0,United Kingdom,0.0
32916,492079,85042,ANTIQUE LILY FAIRY LIGHTS,8,2009-12-15 13:49:00,0.0,15070.0,United Kingdom,0.0
...,...,...,...,...,...,...,...,...,...
979924,575579,22437,SET OF 9 BLACK SKULL BALLOONS,20,2011-11-10 11:49:00,0.0,13081.0,United Kingdom,0.0
979925,575579,22089,PAPER BUNTING VINTAGE PAISLEY,24,2011-11-10 11:49:00,0.0,13081.0,United Kingdom,0.0
1004540,577129,22464,HANGING METAL HEART LANTERN,4,2011-11-17 19:52:00,0.0,15602.0,United Kingdom,0.0
1006110,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,2,2011-11-18 13:23:00,0.0,12444.0,Norway,0.0


*"What do we do with the 'free' transactions? Would it affect the ML modelling?"*

We choose to not include the 'free' transactions since it doesn't add any value to the RFM analysis.

In [36]:
combined_dfs = combined_dfs[combined_dfs['Price'] != 0.0]

### `Country`: Country Check

In [37]:
# Finding the latest date for data-viz :
combined_dfs[combined_dfs['Country'] == 'RSA'].sort_values('InvoiceDate', ascending=False)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
920989,571035,47591D,PINK FAIRY CAKE CHILDRENS APRON,8,2011-10-13 12:50:00,1.95,12446.0,RSA,15.6
920946,571035,23389,SPACEBOY MINI BACKPACK,4,2011-10-13 12:50:00,4.15,12446.0,RSA,16.6
920958,571035,48138,DOORMAT UNION FLAG,2,2011-10-13 12:50:00,8.25,12446.0,RSA,16.5
920957,571035,23395,BELLE JARDINIERE CUSHION COVER,4,2011-10-13 12:50:00,3.75,12446.0,RSA,15.0
920956,571035,22666,RECIPE BOX PANTRY YELLOW DESIGN,6,2011-10-13 12:50:00,2.95,12446.0,RSA,17.7
...,...,...,...,...,...,...,...,...,...
80094,496703,21207,SKULL AND CROSSBONES GARLAND,12,2010-02-03 13:22:00,1.65,12396.0,RSA,19.8
80093,496703,20974,12 PENCILS SMALL TUBE SKULL,24,2010-02-03 13:22:00,0.65,12396.0,RSA,15.6
80092,496703,22329,ROUND CONTAINER SET OF 5 RETROSPOT,12,2010-02-03 13:22:00,1.65,12396.0,RSA,19.8
80091,496703,22090,PAPER BUNTING RETRO SPOTS,6,2010-02-03 13:22:00,2.95,12396.0,RSA,17.7


In [38]:
combined_dfs['Country'].value_counts()

Country
United Kingdom          739994
Germany                  16924
EIRE                     15947
France                   13689
Netherlands               5022
Spain                     3698
Switzerland               3004
Belgium                   2972
Portugal                  2431
Australia                 1901
Channel Islands           1652
Italy                     1499
Norway                    1421
Sweden                    1286
Cyprus                    1171
Finland                    996
Austria                    901
Denmark                    792
Greece                     658
Japan                      578
USA                        530
Poland                     529
Unspecified                522
United Arab Emirates       383
Singapore                  332
Israel                     324
Malta                      291
Iceland                    253
Canada                     227
Lithuania                  189
RSA                        122
Brazil                      94


In [39]:
# ~Personal~ Moral, preference. This author stands firm towards the resistance of colonialism.
combined_dfs = combined_dfs[combined_dfs['Country'] != 'Israel']

In [40]:
combined_dfs

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
1067365,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


Some countries have different terms--take for instance `EIRE` for Ireland, or `West Indies` for South Africa, but we'll just keep the data for now.

### `revenue` : Outlier Check

For the purpose of data visualization and feature engineering, we want to weed out the big returns--To set foot, we're talking returns that add up to more than $2,000.

In [54]:
combined_dfs.sort_values(['revenue', 'Customer ID'], ascending=False)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
748132,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.500000,15098,United Kingdom,38970.00
432176,530715,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,9360,2010-11-04 11:36:00,1.690000,15838,United Kingdom,15818.40
228042,511465,15044A,PINK PAPER PARASOL,3500,2010-06-08 12:59:00,2.550000,18008,United Kingdom,8925.00
873786,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,2011-09-20 11:05:00,5.060000,17450,United Kingdom,7144.72
686007,550461,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-04-18 13:20:00,2.100000,15749,United Kingdom,6539.40
...,...,...,...,...,...,...,...,...,...
846041,C565044,22191,IVORY DINER WALL CLOCK,-318,2011-08-31 17:02:00,7.650000,12931,United Kingdom,-2432.70
267589,C515299,84078A,SET/4 WHITE RETRO STORAGE CUBES,-85,2010-07-09 15:26:00,34.950001,13734,United Kingdom,-2970.75
418860,C529352,71477,COLOUR GLASS. STAR T-LIGHT HOLDER,-1152,2010-10-28 09:32:00,2.750000,17450,United Kingdom,-3168.00
93676,C498151,85220,SMALL FAIRY CAKE FRIDGE MAGNETS,-2504,2010-02-17 10:37:00,1.450000,13902,Denmark,-3630.80


In [42]:
combined_dfs.sort_values(['revenue', 'Customer ID'])[:15]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
685606,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,2011-04-18 13:08:00,2.10,15749.0,United Kingdom,-6539.40
685604,C550456,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1930,2011-04-18 13:08:00,2.55,15749.0,United Kingdom,-4921.50
685602,C550456,48185,DOORMAT FAIRY CAKE,-670,2011-04-18 13:08:00,6.75,15749.0,United Kingdom,-4522.50
875211,C567527,23113,PANTRY CHOPPING BOARD,-756,2011-09-21 09:16:00,5.06,17450.0,United Kingdom,-3825.36
685605,C550456,21175,GIN + TONIC DIET METAL SIGN,-2000,2011-04-18 13:08:00,1.85,15749.0,United Kingdom,-3700.00
93676,C498151,85220,SMALL FAIRY CAKE FRIDGE MAGNETS,-2504,2010-02-17 10:37:00,1.45,13902.0,Denmark,-3630.80
685603,C550456,47566B,TEA TIME PARTY BUNTING,-1300,2011-04-18 13:08:00,2.55,15749.0,United Kingdom,-3315.00
418860,C529352,71477,COLOUR GLASS. STAR T-LIGHT HOLDER,-1152,2010-10-28 09:32:00,2.75,17450.0,United Kingdom,-3168.00
267589,C515299,84078A,SET/4 WHITE RETRO STORAGE CUBES,-85,2010-07-09 15:26:00,34.95,13734.0,United Kingdom,-2970.75
846041,C565044,22191,IVORY DINER WALL CLOCK,-318,2011-08-31 17:02:00,7.65,12931.0,United Kingdom,-2432.70


In [43]:
combined_dfs[combined_dfs['Customer ID'] == 15749]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
578170,540815,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1930,2011-01-11 12:55:00,2.55,15749.0,United Kingdom,4921.5
578171,540815,21175,GIN + TONIC DIET METAL SIGN,2000,2011-01-11 12:55:00,1.85,15749.0,United Kingdom,3700.0
578172,540815,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-01-11 12:55:00,2.10,15749.0,United Kingdom,6539.4
578232,540818,47556B,TEA TIME TEA TOWELS,1300,2011-01-11 12:57:00,2.55,15749.0,United Kingdom,3315.0
578233,540818,48185,DOORMAT FAIRY CAKE,670,2011-01-11 12:57:00,6.75,15749.0,United Kingdom,4522.5
685602,C550456,48185,DOORMAT FAIRY CAKE,-670,2011-04-18 13:08:00,6.75,15749.0,United Kingdom,-4522.5
685603,C550456,47566B,TEA TIME PARTY BUNTING,-1300,2011-04-18 13:08:00,2.55,15749.0,United Kingdom,-3315.0
685604,C550456,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1930,2011-04-18 13:08:00,2.55,15749.0,United Kingdom,-4921.5
685605,C550456,21175,GIN + TONIC DIET METAL SIGN,-2000,2011-04-18 13:08:00,1.85,15749.0,United Kingdom,-3700.0
685606,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,2011-04-18 13:08:00,2.10,15749.0,United Kingdom,-6539.4


Seems like it all come from one customer. And based on the transaction history, he cancels the items after 3 months just to buy another set 12 minutes later.

By this, We'll just clean them out and keep the latest invoice.

In [44]:
invoices_to_clean = ['540815', 'C550456']

combined_dfs = combined_dfs.loc[~(combined_dfs['Invoice'].isin(invoices_to_clean))]

In [45]:
combined_dfs.loc[combined_dfs['Customer ID'] == 15749]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,revenue
578232,540818,47556B,TEA TIME TEA TOWELS,1300,2011-01-11 12:57:00,2.55,15749.0,United Kingdom,3315.0
578233,540818,48185,DOORMAT FAIRY CAKE,670,2011-01-11 12:57:00,6.75,15749.0,United Kingdom,4522.5
686003,550461,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1930,2011-04-18 13:20:00,2.40,15749.0,United Kingdom,4632.0
686004,550461,47556B,TEA TIME TEA TOWELS,1300,2011-04-18 13:20:00,2.10,15749.0,United Kingdom,2730.0
686005,550461,48185,DOORMAT FAIRY CAKE,670,2011-04-18 13:20:00,6.35,15749.0,United Kingdom,4254.5
686006,550461,21175,GIN + TONIC DIET METAL SIGN,2000,2011-04-18 13:20:00,1.69,15749.0,United Kingdom,3380.0
686007,550461,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-04-18 13:20:00,2.10,15749.0,United Kingdom,6539.4


## Fur Future Purposes: Storage Optimization

The data has roughly a million rows--the bigger the data the larger the size. For future uses on data production, we can get good chunks of bytes down by changing the datatypes; the type of data that how it fills the space.

In [46]:
combined_dfs.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
revenue        float64
dtype: object

In [47]:
memory_before = combined_dfs.memory_usage(deep=True)
memory_before

Index           6563376
Invoice        51704680
StockCode      50957413
Description    68741102
Quantity        6563376
InvoiceDate    62352072
Price           6563376
Customer ID     6563376
Country        57665966
revenue         6563376
dtype: int64

In [48]:
combined_dfs.nunique()

Invoice        44007
StockCode       4635
Description     5286
Quantity         631
InvoiceDate    40734
Price            606
Customer ID     5871
Country           40
revenue         5112
dtype: int64

In [49]:
# Scaling the data types down for more size in space:

combined_dfs['Country'] = combined_dfs['Country'].astype('category')

combined_dfs['Customer ID'] = combined_dfs['Customer ID'].astype(int)
combined_dfs['Customer ID'] = (
    pd.to_numeric(combined_dfs['Customer ID'], downcast='unsigned'))

combined_dfs['Price'] = (
    pd.to_numeric(combined_dfs['Price'], downcast='float'))

combined_dfs['Quantity'] = (
    pd.to_numeric(combined_dfs['Quantity'], downcast='integer'))

## Changing InvoiceDate to date format for future data wrangling:
combined_dfs['InvoiceDate'] = pd.to_datetime(combined_dfs['InvoiceDate'])

C:\Users\acer\AppData\Local\Temp\ipykernel_18284\300751000.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_dfs['Country'] = combined_dfs['Country'].astype('category')
C:\Users\acer\AppData\Local\Temp\ipykernel_18284\300751000.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_dfs['Customer ID'] = combined_dfs['Customer ID'].astype(int)
C:\Users\acer\AppData\Local\Temp\ipykernel_18284\300751000.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [50]:
combined_dfs.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int16
InvoiceDate    datetime64[ns]
Price                 float32
Customer ID            uint16
Country              category
revenue               float64
dtype: object

In [51]:
memory_after = combined_dfs.memory_usage(deep=True)
memory_after

Index           6563376
Invoice        51704680
StockCode      50957413
Description    68741102
Quantity        1640844
InvoiceDate     6563376
Price           3281688
Customer ID     1640844
Country          824100
revenue         6563376
dtype: int64

In [52]:
reduction = 100 - (memory_after.sum() / memory_before.sum() * 100)
print(f'File reduction: {reduction:0.3f} %')

File reduction: 38.785 %


Last but not least, we'll end the data cleaning by saving the dataset into a new csv.

In [53]:
combined_dfs.to_csv('online_retail_II_cleaned.csv', index=False)